In [1]:
import torch
import random
from torch.utils import data
from d2l import torch as d2l

In [2]:
true_w = torch.tensor([5.0, 5.0])
true_b = 5.0
features, labels = d2l.synthetic_data(true_w, true_b, 1000)    # true_w为参数，true_b为偏差

In [3]:
# def load_array(data_arrays, batch_size, is_train=True):
#     # 对于给定的tensor数据(样本和标签),将它们包装成dataset
#     # 也就是将训练数据的特征和标签组合
#     dataset = data.TensorDataset(*data_arrays)    
    
#     return data.DataLoader(dataset, batch_size, shuffle=is_train)

def load_array(features, labels, batch_size, is_shuffle):
    # 将训练数据中的特征和标签组合
    dataset = data.TensorDataset(features, labels)
    
    # DataLoader数据加载器，组合数据集和采样器，并在数据集上提供单线程或多线程迭代器
    # 对dataset作进一步的设置
    return data.DataLoader(dataset, batch_size, shuffle=is_shuffle)

In [4]:
batch_size=10
# data_iter = load_array((features, labels), batch_size)
data_iter = load_array(features, labels, batch_size, True)

In [5]:
from torch import nn    # nn为神经网络的缩写

# nn.Sequential()为一个有序的容器
# 神经网络模块将按照在传入构造器的顺序依次被添加到计算图中执行，同时以神经网络模块为元素的有序字典也可以作为传入参数

# nn.Linear(x, y, bias=True)
# x为输入的神经元个数，y为输出的神经元个数，考虑偏置
# 该函数实现功能：制造出一个全连接层的框架，即y = X * W.T + b
net = nn.Sequential(nn.Linear(2, 1))

In [6]:
print(net)    # 可以看到在Sequential中第0层为全连接层，输入两个神经元，输出两个神经元，有偏置

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)


In [7]:
net[0].weight    # 随机给权重值

Parameter containing:
tensor([[-0.1468, -0.5505]], requires_grad=True)

In [8]:
net[0].bias    # 随机给偏置值

Parameter containing:
tensor([0.3911], requires_grad=True)

In [9]:
net[0].weight.data = torch.tensor([[100.0, -100.0]])
net[0].bias.data = torch.tensor([100.])
net[0].weight, net[0].bias

(Parameter containing:
 tensor([[ 100., -100.]], requires_grad=True), Parameter containing:
 tensor([100.], requires_grad=True))

In [10]:
# 均方误差函数
loss = nn.MSELoss()
# a = torch.tensor([5.0]).reshape(1, 1)
# b = torch.tensor([8.0]).reshape(1, 1)
# a, b, loss(a, b)

In [11]:
# torch.optim为一个实现了各种优化算法的库
trainer = torch.optim.SGD(net[0].parameters(), lr=0.003)

In [12]:
num_epoch = 30
for epoch in range(num_epoch):
    for X, y in data_iter:
        l = loss(net[0](X), y)
        trainer.zero_grad()   # 把前一次计算的梯度清零
        l.backward()          # pytorch中已经给我们自动计算sum()，不用我们自己l.sum()了
        trainer.step()        # 进行梯度的更新:w -= lr * ( d( loss(w, b) ) / d( w ) )
    l = loss(net[0](features), labels)
    print(f'epoch {epoch + 1} : w {net[0].weight.data} b {net[0].bias.data}')

epoch 1 : w tensor([[ 55.3043, -51.0336]]) b tensor([57.7154])
epoch 2 : w tensor([[ 31.6078, -24.9339]]) b tensor([34.2498])
epoch 3 : w tensor([[ 19.0642, -11.0066]]) b tensor([21.2264])
epoch 4 : w tensor([[12.4301, -3.5637]]) b tensor([13.9954])
epoch 5 : w tensor([[8.9176, 0.4092]]) b tensor([9.9863])
epoch 6 : w tensor([[7.0618, 2.5376]]) b tensor([7.7657])
epoch 7 : w tensor([[6.0858, 3.6774]]) b tensor([6.5317])
epoch 8 : w tensor([[5.5708, 4.2885]]) b tensor([5.8482])
epoch 9 : w tensor([[5.2996, 4.6167]]) b tensor([5.4696])
epoch 10 : w tensor([[5.1573, 4.7932]]) b tensor([5.2598])
epoch 11 : w tensor([[5.0823, 4.8881]]) b tensor([5.1437])
epoch 12 : w tensor([[5.0431, 4.9393]]) b tensor([5.0793])
epoch 13 : w tensor([[5.0227, 4.9668]]) b tensor([5.0437])
epoch 14 : w tensor([[5.0120, 4.9817]]) b tensor([5.0240])
epoch 15 : w tensor([[5.0064, 4.9897]]) b tensor([5.0130])
epoch 16 : w tensor([[5.0035, 4.9941]]) b tensor([5.0070])
epoch 17 : w tensor([[5.0020, 4.9964]]) b tenso